# Answering Business Questions Using SQL

In this project, we'll analyze data from a SQLite database file called **chinook.db**. This database contains information about a fictional digital music shop divided in eleven tables:

- **employee**
- **customer**
- **invoice**
- **invoice_line**
- **playlist**
- **playlist_track**
- **track**
- **artist**
- **album**
- **media_type**
- **genre**

## Connecting Database to Jupyter Notebook

We'll start by connecting our Jupyter Notebook to our database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

To get an overview of the data, we'll write a query that returns information of all the tables and views in our database file:

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type IN ("table","view");

Done.


type,name,tbl_name,rootpage,sql
table,album,album,2,"CREATE TABLE [album]( [album_id] INTEGER PRIMARY KEY NOT NULL, [title] NVARCHAR(160) NOT NULL, [artist_id] INTEGER NOT NULL, FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,artist,artist,3,"CREATE TABLE [artist]( [artist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,customer,customer,4,"CREATE TABLE [customer]( [customer_id] INTEGER PRIMARY KEY NOT NULL, [first_name] NVARCHAR(40) NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [company] NVARCHAR(80), [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60) NOT NULL, [support_rep_id] INTEGER, FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,employee,employee,5,"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,genre,genre,6,"CREATE TABLE [genre]( [genre_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,invoice,invoice,7,"CREATE TABLE [invoice]( [invoice_id] INTEGER PRIMARY KEY NOT NULL, [customer_id] INTEGER NOT NULL, [invoice_date] DATETIME NOT NULL, [billing_address] NVARCHAR(70), [billing_city] NVARCHAR(40), [billing_state] NVARCHAR(40), [billing_country] NVARCHAR(40), [billing_postal_code] NVARCHAR(10), [total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([customer_id]) REFERENCES [customer] ([customer_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,invoice_line,invoice_line,8,"CREATE TABLE [invoice_line]( [invoice_line_id] INTEGER PRIMARY KEY NOT NULL, [invoice_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, [unit_price] NUMERIC(10,2) NOT NULL, [quantity] INTEGER NOT NULL, FOREIGN KEY ([invoice_id]) REFERENCES [invoice] ([invoice_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,media_type,media_type,9,"CREATE TABLE [media_type]( [media_type_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist,playlist,10,"CREATE TABLE [playlist]( [playlist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist_track,playlist_track,11,"CREATE TABLE [playlist_track]( [playlist_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, CONSTRAINT [pk_playlist_track] PRIMARY KEY ([playlist_id], [track_id]), FOREIGN KEY ([playlist_id]) REFERENCES [playlist] ([playlist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


The table above shows us the 11 tables and the columns for each table.

## Determining more popular genres in USA

Assuming that the Chinook is going to advertise new albuns in the USA.
We've been tasked with selecting the first three albums that will be added to the store, from the next list of artists:

- Regal, Hip-Hop
- Red Tone, Punk
- Meteor and the Girls, Pop
- Slim Jim Bites, Blues

All this artist don't have any tracks in the store right now, so we only have the artist name and the genre of music.

To know which one of the albuns we are going to select, we'll write a query to find out which genres sell the most tracks in the USA and then make a recommendation for the three artists. Firstly, we'll analyse the columns in the invoice table and check in which column appears USA.

In [3]:
%%sql
SELECT *
  FROM invoice
 LIMIT 10;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83
6,31,2017-01-10 00:00:00,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,1.98
7,49,2017-01-12 00:00:00,Ordynacka 10,Warsaw,None,Poland,00-358,10.89
8,59,2017-01-13 00:00:00,"3,Raj Bhavan Road",Bangalore,None,India,560001,9.9
9,18,2017-01-18 00:00:00,627 Broadway,New York,NY,USA,10012-2612,8.91
10,31,2017-01-18 00:00:00,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,1.98


We can see that the USA appears in the BillingCountry.

In [4]:
%%sql

WITH usa_data AS 
              (
               SELECT *
               FROM track t
               INNER JOIN invoice_line il ON il.track_id = t.track_id
               INNER JOIN invoice i ON i.invoice_id = il.invoice_id
               INNER JOIN customer c ON c.customer_id = i.customer_id
               WHERE c.country = 'USA'
              )
    
SELECT g.name genre,
       usa_data.country country,
       COUNT(g.name) count,
       cast(COUNT(g.name) AS FLOAT) / (SELECT COUNT(*)
                                              FROM usa_data) percentage
FROM usa_data
INNER JOIN genre g ON g.genre_id = usa_data.genre_id
WHERE usa_data.country = 'USA'
GROUP BY genre
ORDER BY count DESC
LIMIT 10;

Done.


genre,country,count,percentage
Rock,USA,561,0.5337773549000951
Alternative & Punk,USA,130,0.12369172216936251
Metal,USA,124,0.11798287345385347
R&B/Soul,USA,53,0.05042816365366318
Blues,USA,36,0.03425309229305423
Alternative,USA,35,0.03330161750713606
Latin,USA,22,0.02093244529019981
Pop,USA,22,0.02093244529019981
Hip Hop/Rap,USA,20,0.019029495718363463
Jazz,USA,14,0.013320647002854425


We can observe that the genres in the list above that have more number tracks sold in the USA are Punk, Blues and Pop so the first three albums that should be added to the store are: Red Tone, Slim Jim Bites and Meteor and the Girls. 

## Analyzing employee performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others. 
We will find the total dollar amount of sales assigned to each sales support agent within the company.

In [5]:
%%sql

SELECT e.first_name || ' ' || e.last_name AS employee,
       e.hire_date,
       COUNT(i.customer_id) AS number_customers,
       CAST(SUM(i.total) AS INT) AS total_sales
FROM customer c
INNER JOIN employee e ON e.employee_id = c.support_rep_id
INNER JOIN invoice i ON i.customer_id = c.customer_id
INNER JOIN invoice_line il ON il.invoice_line_id = i.invoice_id
GROUP BY 1;

Done.


employee,hire_date,number_customers,total_sales
Jane Peacock,2017-04-01 00:00:00,212,1731
Margaret Park,2017-05-03 00:00:00,214,1584
Steve Johnson,2017-10-17 00:00:00,188,1393


We can see that Jane Peacock has the highest total dollar amount of sales assigned. There's a difference of 19,5\% between the total invoices of Jane Peacock and Steve Johnson. However, this difference is very small comparing with the diffrences between the hiring dates.

## Analyzing sales data for customers from different countries

Your next task is to analyze the sales data for customers from each different country. You have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:

- total number of customers 
- total value of sales
- average value of sales per customer
- average order value


In [6]:
%%sql

DROP VIEW if exists teste;

CREATE VIEW teste AS
    SELECT
        CASE
            WHEN (
                SELECT COUNT(*)
                FROM customer
                WHERE country = c.country
            ) = 1 THEN "Other"
            ELSE c.country
            END
        AS country,
        c.customer_id,
        il.unit_price,
        i.invoice_id
    FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
    ORDER BY i.total ASC;

Done.
Done.


[]

In [7]:
%%sql
SELECT
    country,
    COUNT(distinct customer_id) number_customers,
    ROUND(SUM(unit_price), 2) total_sales,
    ROUND(SUM(unit_price) / count(distinct customer_id),2) avg_value_sales,
    ROUND(SUM(unit_price) / count(distinct invoice_id),2) average_order
FROM (
    SELECT
        teste.*,
        CASE
            WHEN teste.country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM teste
)
GROUP BY country
ORDER BY sort ASC, 2 DESC;

Done.


country,number_customers,total_sales,avg_value_sales,average_order
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
United Kingdom,3,245.52,81.84,8.77
Czech Republic,2,273.24,136.62,9.11
India,2,183.15,91.58,8.72
Portugal,2,185.13,92.57,6.38
Other,15,1094.94,73.0,7.45


Based on the results, we can conclude that the USA is by far the most important country for sales. When we look at the average order value, there may be opportunities in the Czech Republic, United Kingdom and India.